In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [3]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random

In [4]:
%run ../automated_model_creation/MODEL_LIST.ipynb            #Load the models

In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../event_scraper/scraped_event.csv')

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4211
4211
4211


In [12]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Tyron Woodley,Gilbert Burns,-170.0,150.0,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,2.54,-7.62,-5.0,0.630000,0.200000,1.180000,1.0,1.0,1
1,Blagoy Ivanov,Augusto Sakai,-105.0,-115.0,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,10.16,10.16,-4.0,2.420000,-0.200000,-0.830000,1.0,1.0,1
2,Billy Quarantillo,Spike Carlyle,-130.0,110.0,76.923077,110.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,2.54,-4.0,3.350000,-4.000000,-0.800000,1.0,1.0,0
3,Roosevelt Roberts,Brok Weaver,-360.0,300.0,27.777778,300.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,0.00,2.0,2.700000,0.300000,-1.840000,1.0,1.0,0
4,Mackenzie Dern,Hannah Cifers,-420.0,335.0,23.809524,335.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-7.62,-2.54,0.0,1.580000,-0.600000,-0.140000,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4280,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4281,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4282,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Amanda Nunes,Felicia Spencer,-600.0,450.0,16.666667,450.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,-2.54,-3.0,0.20,0.0,-0.97,1.0,1.0,0
1,Raphael Assuncao,Cody Garbrandt,120.0,-140.0,120.000000,71.428571,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,7.62,-2.54,-9.0,0.21,-0.3,-0.60,1.0,1.0,0
2,Aljamain Sterling,Cory Sandhagen,-115.0,-105.0,86.956522,95.238095,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,10.16,-2.54,-2.0,2.29,-0.2,-0.67,1.0,1.0,0
3,Neil Magny,Anthony Rocco Martin,-135.0,115.0,74.074074,115.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,2.0,-7.62,-17.78,-2.0,-1.23,0.8,-2.02,1.0,1.0,0
4,Eddie Wineland,Sean O'Malley,400.0,-500.0,400.000000,20.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,0.0,10.16,7.62,-10.0,3.46,1.1,0.96,1.0,1.0,0
5,Alex Caceres,Chase Hooper,168.0,-195.0,168.000000,51.282051,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,7.62,2.54,-11.0,1.18,3.0,0.24,1.0,1.0,0
6,Ian Heinisch,Gerald Meerschaert,-130.0,110.0,76.923077,110.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,5.0,5.08,12.70,1.0,-0.06,1.2,1.19,1.0,1.0,0
7,Cody Stamann,Brian Kelleher,-250.0,210.0,40.000000,210.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,2.0,0.00,5.08,3.0,0.40,0.8,-2.32,1.0,1.0,0
8,Charles Byrd,Maki Pitolo,-175.0,155.0,57.142857,155.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,0.00,5.08,-7.0,1.56,-1.0,-2.52,1.0,1.0,0
9,Jussier Formiga,Alex Perez,115.0,-135.0,115.000000,74.074074,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,2.54,-5.08,-7.0,3.13,0.9,1.40,1.0,1.0,0


In [13]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        if t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        
        
        print()
        
    return(0)
        

In [14]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    
    return(0)

In [15]:

get_bets(model_4, features_4, min_ev_4, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Amanda Nunes vs. Felicia Spencer
Amanda Nunes has an EV of 1.81 on odds of -600.0.  They have 87.27 % chance of winning.

Raphael Assuncao vs. Cody Garbrandt
Raphael Assuncao has an EV of 5.59 on odds of 120.0.  They have 47.99 % chance of winning.

Aljamain Sterling vs. Cory Sandhagen
Aljamain Sterling has an EV of 1.6 on odds of -115.0.  They have 54.34 % chance of winning.

Neil Magny vs. Anthony Rocco Martin

Eddie Wineland vs. Sean O'Malley

Alex Caceres vs. Chase Hooper
Alex Caceres has an EV of 9.51 on odds of 168.0.  They have 40.86 % chance of winning.

Ian Heinisch vs. Gerald Meerschaert
Gerald Meerschaert has an EV of 0.41 on odds of 110.0.  They have 47.81 % chance of winning.

Cody Stamann vs. Brian Kelleher
Cody Stamann has an EV of 0.93 on odds of -250.0.  They have 72.09 % chance of winning.

Charles Byrd vs. Maki Pitolo
Maki Pitolo has an EV of 10.54 on odds of 155.0.  They have 43.35 % chance of winning.

Jussier Formiga vs. Alex Perez
Jussier Formiga has an EV of 9.6

0